In [ ]:
# Importing the libraries
from __future__ import print_function
import shap
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
import os

: 

In [ ]:
print(tf.version)

In [ ]:
# Proprocessing the test and training set
train_datagen = ImageDataGenerator(rescale = 1./255, #pixelwaarden normaliseren
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32, #hoeveel foto's per keer langs neural netwerk laat passeren
                                                 class_mode = 'binary')
#fit from generator
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [ ]:
# Initialising the CNN
cnn = tf.keras.models.Sequential() # layers gaan op elkaar volgen

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))

# Flattening
cnn.add(tf.keras.layers.Flatten()) # full connection maken (zorgt voor features)

# Full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # sigmoid = input transformeren naar 0-1

# Compile CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

cnn.fit(x = training_set, validation_data = test_set, epochs = 1)

In [ ]:
# Making a prediction
test_image = image.load_img('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing/waldo/12_2_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  print("Tested negative for Waldo :'(")
else:
  print('Tested positive for Waldo! :D')

In [ ]:
x_train, y_train = next(training_set)
x_test, y_test = next(test_set)
#model.fit(x = training_set, validation_data = test_set, epochs = 1)
score = cnn.evaluate(test_set, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=True)]

In [ ]:
# explain predictions of the model (named cnn in this case) on three images
#e = shap.DeepExplainer(cnn, background)
e = shap.DeepExplainer((cnn.layers[0].input, cnn.layers[-1].output), background)

In [ ]:
shap_values = e.shap_values(x_test[1:5][1:5])
print(shap_values)